In [29]:
import pandas as pd 
import numpy as np 
import glob
import matplotlib.pyplot as plt
from time import time

from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, recall_score
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold

# Differnt types of Naive Bayes Classifiers
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

pd.set_option('display.max_columns', None)

In [31]:
df = pd.read_csv('../twitter_data_train_multiclass.csv')
test = pd.read_csv('../twitter_data_test_multiclass.csv')

In [32]:
df.head()

,Unnamed: 0,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,default_profile,default_profile_image,geo_enabled,profile_image_url,profile_use_background_image,profile_background_tile,protected,verified,description,account_type,tweets_list,tweet_frequency,number_of_tags,number_of_mentions,tweet_weekend_frequency,tweet_weekday_frequency,following_to_followers_ratio,username_length,screen_name_length,username_spec_char_count,screen_name_spec_char_count,has_url,has_desc,tweets_list_processed,description_processed,tweets_0,tweets_1,tweets_2,tweets_3,tweets_4,tweets_5,tweets_6,tweets_7,tweets_8,tweets_9,tweets_10,tweets_11,tweets_12,tweets_13,tweets_14,tweets_15,tweets_16,tweets_17,tweets_18,tweets_19,tweets_20,tweets_21,tweets_22,tweets_23,tweets_24,tweets_25,tweets_26,tweets_27,tweets_28,tweets_29,tweets_30,tweets_31,tweets_32,tweets_33,tweets_34,tweets_35,tweets_36,tweets_37,tweets_38,tweets_39,tweets_40,tweets_41,tweets_42,tweets_43,tweets_44,tweets_45,tweets_46,tweets_47,tweets_48,tweets_49,tweets_50,tweets_51,tweets_52,tweets_53,tweets_54,tweets_55,tweets_56,tweets_57,tweets_58,tweets_59,tweets_60,tweets_61,tweets_62,tweets_63,tweets_64,tweets_65,tweets_66,tweets_67,tweets_68,tweets_69,tweets_70,tweets_71,tweets_72,tweets_73,tweets_74,tweets_75,tweets_76,tweets_77,tweets_78,tweets_79,tweets_80,tweets_81,tweets_82,tweets_83,tweets_84,tweets_85,tweets_86,tweets_87,tweets_88,tweets_89,tweets_90,tweets_91,tweets_92,tweets_93,tweets_94,tweets_95,tweets_96,tweets_97,tweets_98,tweets_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,has_face,reciprocity,account_type_multi
0,0,82118682,davide gazzÃ¨,davidegazze,88,19,39,9,0,NaN,0.0,0.0,0.0,http://a0.twimg.com/profile_images/1700682774/...,1.0,1.0,NaN,NaN,NaN,1,['RT @myself2048: @beppe_grillo @pbersani @ber...,1.491525,40.0,38.0,1.142857,1.600000,2.052632,13,11,2,0,0,0,myself2048 beppe_grillo pbersani berlusconi20...,NaN,0.091977,0.034554,0.000000,0.059238,0.000000,0.000000,0.000000,0.056452,0.048987,0.058207,0.11607,0.125471,0.0,0.000000,0.151018,0.000000,0.000000,0.058498,0.15312,0.118353,0.439767,0.000000,0.200249,0.0,0.000000,0.178702,0.030877,0.000000,0.000000,0.000000,0.000000,0.0,0.030396,0.0,0.0,0.386811,0.028850,0.0,0.111074,0.000000,0.069019,0.071754,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.031558,0.000000,0.000000,0.000000,0.0,0.000000,0.063091,0.000000,0.000000,0.161387,0.059978,0.000000,0.294536,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.083316,0.000000,0.0,0.000000,0.136500,0.0,0.

## Get to the correct data format

In [33]:
to_drop = ['Unnamed: 0', 'screen_name', 'url', 'profile_image_url', 'description',
           'id', 'name', 'account_type', 'tweets_list', 'tweets_list_processed',
          'description_processed', 'protected', 'verified', 'account_type_multi',
          'profile_background_tile', 'profile_use_background_image']

X_train, y_train = df.drop(to_drop, axis=1), df['account_type_multi']

In [34]:
X_test, y_test = test.drop(to_drop, axis=1), test['account_type_multi']

In [35]:
X_train.head()

,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,geo_enabled,tweet_frequency,number_of_tags,number_of_mentions,tweet_weekend_frequency,tweet_weekday_frequency,following_to_followers_ratio,username_length,screen_name_length,username_spec_char_count,screen_name_spec_char_count,has_url,has_desc,tweets_0,tweets_1,tweets_2,tweets_3,tweets_4,tweets_5,tweets_6,tweets_7,tweets_8,tweets_9,tweets_10,tweets_11,tweets_12,tweets_13,tweets_14,tweets_15,tweets_16,tweets_17,tweets_18,tweets_19,tweets_20,tweets_21,tweets_22,tweets_23,tweets_24,tweets_25,tweets_26,tweets_27,tweets_28,tweets_29,tweets_30,tweets_31,tweets_32,tweets_33,tweets_34,tweets_35,tweets_36,tweets_37,tweets_38,tweets_39,tweets_40,tweets_41,tweets_42,tweets_43,tweets_44,tweets_45,tweets_46,tweets_47,tweets_48,tweets_49,tweets_50,tweets_51,tweets_52,tweets_53,tweets_54,tweets_55,tweets_56,tweets_57,tweets_58,tweets_59,tweets_60,tweets_61,tweets_62,tweets_63,tweets_64,tweets_65,tweets_66,tweets_67,tweets_68,tweets_69,tweets_70,tweets_71,tweets_72,tweets_73,tweets_74,tweets_75,tweets_76,tweets_77,tweets_78,tweets_79,tweets_80,tweets_81,tweets_82,tweets_83,tweets_84,tweets_85,tweets_86,tweets_87,tweets_88,tweets_89,tweets_90,tweets_91,tweets_92,tweets_93,tweets_94,tweets_95,tweets_96,tweets_97,tweets_98,tweets_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,has_face,reciprocity
0,88,19,39,9,0,0.0,0.0,0.0,1.491525,40.0,38.0,1.142857,1.600000,2.052632,13,11,2,0,0,0,0.091977,0.034554,0.000000,0.059238,0.000000,0.000000,0.000000,0.056452,0.048987,0.058207,0.11607,0.125471,0.0,0.000000,0.151018,0.000000,0.000000,0.058498,0.15312,0.118353,0.439767,0.000000,0.200249,0.0,0.000000,0.178702,0.030877,0.000000,0.000000,0.000000,0.000000,0.0,0.030396,0.0,0.0,0.386811,0.028850,0.0,0.111074,0.000000,0.069019,0.071754,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.031558,0.000000,0.000000,0.000000,0.0,0.000000,0.063091,0.000000,0.000000,0.161387,0.059978,0.000000,0.294536,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.083316,0.000000,0.0,0.000000,0.136500,0.0,0.000000,0.114215,0.374189,0.000000,0.0,0.000000,0.000000,0.0,0.029388,0.000000,0.177164,0.180592,0.000000,0.267742,0.0,0.091636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

## Mixed NB Approach 1 
train a gaussian NB model on all features

In [36]:
model_G = GaussianNB()
clf_G = model_G.fit(X_train, y_train)
y_pred = model_G.predict(X_test)

In [37]:
# train classification report
y_train_pred = model_G.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.92      0.59      0.72      2338
           1       0.46      0.99      0.63      2706
           2       0.80      0.81      0.81      1494
           3       0.52      0.12      0.20      2908

    accuracy                           0.59      9446
   macro avg       0.68      0.63      0.59      9446
weighted avg       0.65      0.59      0.55      9446



In [38]:
# test classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.58      0.71       413
           1       0.46      0.99      0.62       460
           2       0.81      0.81      0.81       276
           3       0.51      0.13      0.21       518

    accuracy                           0.59      1667
   macro avg       0.68      0.63      0.59      1667
weighted avg       0.65      0.59      0.55      1667



## Mixed NB Approach 2
train three separate models using continuous, binary and categorical variables and then train the final model based on predictions from the first three models

In [39]:
binary_cols = ['default_profile', 'default_profile_image', 'geo_enabled', 'default_profile_image', 'has_url', 'has_face']
continuous_cols = ['statuses_count', 'followers_count', 'friends_count', 'favourites_count', 'listed_count', 'tweet_frequency',    
                    'number_of_tags', 'number_of_mentions', 'tweet_weekend_frequency', 'tweet_weekday_frequency',               
                    'following_to_followers_ratio', 'username_length', 'screen_name_length', 'username_spec_char_count', 
                    'screen_name_spec_char_count']												

In [40]:
train_binary = X_train[binary_cols]
train_continuous = X_train[continuous_cols]
train_text = X_train.drop(binary_cols+continuous_cols, axis=1) 

In [41]:
test_binary = X_test[binary_cols]
test_continuous = X_test[continuous_cols]
test_text = X_test.drop(binary_cols+continuous_cols, axis=1) 

In [42]:
# ----- Fit three models -----

model_G = GaussianNB()
clf_G = model_G.fit(train_continuous, y_train)

model_M = MultinomialNB()
clf_M = model_M.fit(train_text, y_train)

model_B = BernoulliNB()
clf_B = model_B.fit(train_binary, y_train)

# ----- Get probability predictions from each model -----
# On training data
G_train_probas = model_G.predict_proba(train_continuous)
M_train_probas = model_M.predict_proba(train_text)
B_train_probas = model_B.predict_proba(train_binary)

# And on testing data
G_test_probas = model_G.predict_proba(test_continuous)
M_test_probas = model_M.predict_proba(test_text)
B_test_probas = model_B.predict_proba(test_binary)

# Combine probability prediction for class=1 from both models
X_new_train = np.c_[(G_train_probas[:,1], M_train_probas[:,1], B_train_probas[:,1])] # Train
X_new_test = np.c_[(G_test_probas[:,1], M_test_probas[:,1], B_test_probas[:,1])] # Test


# ----- Fit Gaussian model on the X_new -----
model = GaussianNB()
clf = model.fit(X_new_train, y_train)
y_pred = model.predict(X_new_test)

In [43]:
# train classification report
y_train_pred = model.predict(X_new_train)
print(classification_report(y_train, y_train_pred, digits=5))

              precision    recall  f1-score   support

           0    0.59184   0.23567   0.33711      2338
           1    0.94756   0.94161   0.94458      2706
           2    0.40047   0.79987   0.53372      1494
           3    0.83814   0.81912   0.82852      2908

    accuracy                        0.70675      9446
   macro avg    0.69450   0.69907   0.66098      9446
weighted avg    0.73930   0.70675   0.69351      9446



In [44]:
# test classification report
print(classification_report(y_test, y_pred, digits=5))

              precision    recall  f1-score   support

           0    0.57988   0.23729   0.33677       413
           1    0.97297   0.93913   0.95575       460
           2    0.40590   0.79710   0.53790       276
           3    0.81836   0.80888   0.81359       518

    accuracy                        0.70126      1667
   macro avg    0.69428   0.69560   0.66100      1667
weighted avg    0.73365   0.70126   0.68904      1667



## Hyperparameter Tuning

In [45]:
param_grid = {
    'var_smoothing': np.logspace(0,-9, num=100)
}
cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=999)

For approach 1:

In [46]:
model_grid1 = RandomizedSearchCV(estimator = GaussianNB(), 
                            param_distributions = param_grid, 
                            verbose = 1, 
                            cv = cv_method,
                            n_iter = 100,
                            scoring = 'f1_weighted')
model_grid1.fit(X_train, y_train)
print('--------------------------------------------------------')
print('Best Estimator: ', model_grid1.best_estimator_)

Fitting 15 folds for each of 100 candidates, totalling 1500 fits
--------------------------------------------------------
Best Estimator:  GaussianNB()


In [47]:
# train classification report
y_train_pred = model_grid1.predict(X_train)
print(classification_report(y_train, y_train_pred, digits=5))

              precision    recall  f1-score   support

           0    0.92421   0.58939   0.71977      2338
           1    0.46437   0.98965   0.63213      2706
           2    0.80106   0.81124   0.80612      1494
           3    0.52148   0.12105   0.19648      2908

    accuracy                        0.59496      9446
   macro avg    0.67778   0.62783   0.58862      9446
weighted avg    0.64902   0.59496   0.54722      9446



In [48]:
# test classification report
y_pred = model_grid1.predict(X_test)
print(classification_report(y_test, y_pred, digits=5))

              precision    recall  f1-score   support

           0    0.92636   0.57869   0.71237       413
           1    0.45554   0.99130   0.62423       460
           2    0.80866   0.81159   0.81013       276
           3    0.51145   0.12934   0.20647       518

    accuracy                        0.59148      1667
   macro avg    0.67550   0.62773   0.58830      1667
weighted avg    0.64803   0.59148   0.54703      1667



For approach 2:

In [49]:
model_grid2 = RandomizedSearchCV(estimator = GaussianNB(), 
                            param_distributions = param_grid, 
                            verbose = 1, 
                            cv = cv_method,
                            n_iter = 100,
                            scoring = 'f1_weighted')
model_grid2.fit(X_new_train, y_train)
print('--------------------------------------------------------')
print('Best Estimator: ', model_grid2.best_estimator_)

Fitting 15 folds for each of 100 candidates, totalling 1500 fits
--------------------------------------------------------
Best Estimator:  GaussianNB(var_smoothing=0.3511191734215131)


In [50]:
# train classification report
y_train_pred = model_grid2.predict(X_new_train)
print(classification_report(y_train, y_train_pred, digits=5))

              precision    recall  f1-score   support

           0    0.74341   0.73610   0.73974      2338
           1    0.94756   0.94161   0.94458      2706
           2    0.71296   0.61847   0.66237      1494
           3    0.76224   0.82462   0.79220      2908

    accuracy                        0.80362      9446
   macro avg    0.79154   0.78020   0.78472      9446
weighted avg    0.80288   0.80362   0.80233      9446



In [51]:
# test classification report
y_pred = model_grid2.predict(X_new_test)
print(classification_report(y_test, y_pred, digits=5))

              precision    recall  f1-score   support

           0    0.72906   0.71671   0.72283       413
           1    0.95991   0.93696   0.94829       460
           2    0.71429   0.61594   0.66148       276
           3    0.73345   0.81274   0.77106       518

    accuracy                        0.79064      1667
   macro avg    0.78418   0.77059   0.77592      1667
weighted avg    0.79168   0.79064   0.78988      1667



## Best NB Model
Approach 2 has a higher recall score than approach 1.

## Run time

In [28]:
total_time = 0
for i in range(100):
    start_time = time()
    model_time = GaussianNB(var_smoothing=0.03511191734215131)
    clf_best = model_time.fit(X_new_train, y_train)
    y_pred = model_time.predict(X_new_test)
    run_time = time() - start_time
    total_time += run_time
avg_time = total_time / 100
print("Run time: ", avg_time)

Run time:  0.004966552257537842
